# Chapter xx

*Data Structures and Information Retrieval in Python*

Copyright 2021 Allen Downey

License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [120]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
# download('https://github.com/AllenDowney/DSIRP/raw/main/utils.py')

[Click here to run this chapter on Colab](https://colab.research.google.com/github/AllenDowney/DSIRP/blob/main/chapters/chap01.ipynb)

In [121]:
class Node:
    def __init__(self, data, next=None):
        self.data = data
        self.next = next

In [122]:
node1 = Node(1, None)

In [123]:
def node_repr(node):
    if node is None:
        return 'None'
    
    rest = node_repr(node.next)
    return f'Node({node.data}, {rest})'

In [124]:
node_repr(node1)

'Node(1, None)'

In [125]:
Node.__repr__ = node_repr

In [126]:
node1

Node(1, None)

In [127]:
node2 = Node(2)
node3 = Node(3)

And then link them up, like this:

In [128]:
node1.next = node2
node2.next = node3

In [129]:
node1

Node(1, Node(2, Node(3, None)))

In [130]:
class List:
    def __init__(self, head=None):
        self.head = head

In [131]:
t = List(node1)
t

In [132]:
def list_repr(node):
    return f'List({node_repr(node.head)})'

In [133]:
list_repr(t)

'List(Node(1, Node(2, Node(3, None))))'

In [134]:
List.__repr__ = list_repr

In [135]:
t

List(Node(1, Node(2, Node(3, None))))

## Search

In [136]:
def is_in(data, t):
    node = t.head
    while node:
        if node.data == data:
            return True
        node = node.next
    return False

In [137]:
is_in(1, t), is_in(3, t), is_in(5, t)

(True, True, False)

In [138]:
def is_in_helper(node, data):
    if node is None:
        return False
    if node.data == data:
        return True
    return is_in_helper(node.next, data)

def is_in(data, t):
    return is_in_helper(t.head, data)

In [139]:
is_in(1, t), is_in(3, t), is_in(5, t)

(True, True, False)

## Total

In [143]:
def list_total(t):
    total = 0
    node = t.head
    while node:
        total += node.data
        node = node.next
    return total

In [144]:
list_total(t)

6

In [145]:
def list_total_helper(node):
    if node is None:
        return 0
    return node.data + list_total_helper(node.next)

def list_total(t):
    return list_total_helper(t.head)

In [146]:
list_total(t)

6

## Push and Pop

In [164]:
def push(t, node):
    node.next = t.head
    t.head = node

In [165]:
t = List()
push(t, Node(3))
push(t, Node(2))
push(t, Node(1))
t

List(Node(1, Node(2, Node(3, None))))

In [166]:
def pop(t):
    if t.head is None:
        raise ValueError('Tried to pop from empty list')
    data = t.head.data
    t.head = t.head.next
    return data

In [167]:
pop(t), pop(t), pop(t)

(1, 2, 3)

In [168]:
t

List(None)

## Reverse

Based on https://www.geeksforgeeks.org/reverse-a-linked-list/

Simplest if you are allowed to allocate new Nodes

In [187]:
def reverse(t):
    t2 = List()
    node = t.head
    while node:
        push(t2, Node(node.data))
        node = node.next

    return t2

In [188]:
t = List(Node(1, Node(2, Node(3, None))))
reverse(t)

List(Node(3, Node(2, Node(1, None))))

This one allocates an array and a List, but it reuses the Node objects

In [178]:
def reverse(t):
    queue = []
    node = t.head
    while node:
        queue.append(node)
        node = node.next

    t2 = List()
    for node in queue:
        push(t2, node)
        
    t.head = t2.head

In [179]:
t = List(Node(1, Node(2, Node(3, None))))
reverse(t)
t

List(Node(3, Node(2, Node(1, None))))

Here's a recursive version that doesn't allocate anything

In [182]:
def reverse_helper(node):
 
    # if there are 0 or 1 nodes
    if node is None or node.next is None:
        return node

    # reverse the rest list
    rest = reverse_helper(node.next)

    # Put first element at the end
    node.next.next = node
    node.next = None

    return rest

def reverse(t):
    t.head = reverse_helper(t.head)

In [183]:
t = List(Node(1, Node(2, Node(3, None))))
reverse(t)
t

List(Node(3, Node(2, Node(1, None))))

And finally an iterative version that doesn't allocate anything.

In [180]:
def reverse(t):
    prev = None
    current = t.head
    while current :
        next = current.next
        current.next = prev
        prev = current
        current = next
    t.head = prev

## Remove

In [191]:
def remove_after(node):
    node.next = node.next.next

In [192]:
t = List(Node(1, Node(2, Node(3, None))))
remove_after(t.head)
t

List(Node(1, Node(3, None)))

In [200]:
def remove(t, data):
    if t.head is None:
        raise ValueError('Tried to remove from an empty list')
        
    node = t.head
    if node.data == data:
        t.head = node.next
        return

    while node.next:
        if node.next.data == data:
            remove_after(node)
            return
        node = node.next
        
    raise ValueError('Value not found')

In [208]:
t = List(Node(1, Node(2, Node(3, None))))
remove(t, 2)
t

List(Node(1, Node(3, None)))

In [209]:
remove(t, 1)
t

List(Node(3, None))

In [210]:
try:
    remove(t, 4)
except ValueError as e:
    print(e)

Value not found


In [211]:
remove(t, 3)
t

List(None)

In [212]:
try:
    remove(t, 5)
except ValueError as e:
    print(e)

Tried to remove from an empty list


## Insert Sorted

In [213]:
def insert_after(node, data):
    node.next = Node(data, node.next)

In [214]:
t = List(Node(1, Node(2, Node(3, None))))
insert_after(t.head, 5)
t

List(Node(1, Node(5, Node(2, Node(3, None)))))

In [230]:
def insert_sorted(t, data):
    if t.head is None or t.head.data > data:
        push(t, Node(data))
        return
    
    node = t.head
    while node.next:
        if node.next.data > data:
            insert_after(node, data)
            return
        node = node.next
    
    insert_after(node, data)

In [231]:
t = List()
insert_sorted(t, 1)
t

List(Node(1, None))

In [232]:
insert_sorted(t, 3)
t

List(Node(1, Node(3, None)))

In [233]:
insert_sorted(t, 0)
t

List(Node(0, Node(1, Node(3, None))))

In [234]:
insert_sorted(t, 2)
t

List(Node(0, Node(1, Node(2, Node(3, None)))))